<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Object Oriented Programming: Coding a Linear Regression Class

_Authors: Kiefer Katovich (SF)_

---

### Learning Objectives
- Learn the fundamentals of object oriented programming in python
- Review the solution to coefficients for multiple linear regression
- Apply object oriented programming concepts to build a linear regression class by hand

### Lesson Guide
- [Review the linear algebra derivation of coefficients for MLR](#review-mlr)
- [Load the simple housing data](#load-data)
- [Classes and objects](#classes-objects)
- [Coding our own `LinearRegression` class](#coding-lr)
    - [Starting a basic python class](#starting-class)
    - [Adding a class function](#class-function)
    - [Assigning attributes during instantiation](#init-args)
    - [Add another function to add an intercept](#intercept-adder)
    - [Instantiate the class](#instantiate)
    - [Add a predict function](#predict)
    - [Add a score function](#score)
- [Verify your class against the sklearn implementation](#verify)
- [Inspecting a class](#inspection)
- [Some special class methods](#special)

<a id='review-mlr'></a>

## Review: solving for the coefficients that minimize the loss

---

### The "least squares" solution to linear regression

**Step 1:** With target vector $y$ and prediction matrix $X$, we can formulate a regression as:

### $$ y = \beta X + \epsilon $$

Where $\beta$ is our vector of coefficients and $\epsilon$ is our vector of errors, or residuals.

**Step 2:** We can equivalently formulate this as a calculation of the residuals:

### $$ \epsilon = \beta X - y $$

*Our goal is to minimize the sum of squared residuals.* This is also known as the "least squares loss function". 

**Step 3:** Solve for the sum of squared residuals on the left side of the equation. Recall the vector of errors are equivalent to the residuals. The sum of squared residuals is represented as the dot product of the vector of residuals.

### $$ \sum_{i=1}^n \epsilon_i^2 = 
\left[\begin{array}{cc}
\epsilon_1 \cdots \epsilon_n
\end{array}\right] 
\left[\begin{array}{cc}
\epsilon_1 \\ \cdots \\ \epsilon_n
\end{array}\right] = \epsilon' \epsilon
$$

Therefore we can write the sum of squared residuals as:

### $$ \epsilon' \epsilon = (\beta X - y)' (\beta X - y) $$

Which becomes:

### $$ \epsilon' \epsilon = y'y - y'X\beta - \beta' X' y + \beta' X' X \beta $$

**Step 4:** We want to find the coefficients where the loss function will be minimum. In this case we can use calculus, taking the derivative with respect to the $\beta$ vector:

### $$ \frac{\partial \epsilon' \epsilon}{\partial \beta} = 
-2X'y + 2X'X\beta$$

Since want to minimize loss function and the loss function is convex, we set the derivative to zero and solve for the beta coefficient vector:

### $$ 0 = -2X'y + 2X'X\beta \\
X'X\beta = X'y \\
\beta = (X'X)^{-1}X'y$$

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

<a id='load-data'></a>

## Load the simple housing data

---

This dataset only has 4 columns. We can formulate simple regression problems with it to test our linear regression class down the line.

In [2]:
house = './datasets/housing-data.csv'
house = pd.read_csv(house)

<a id='classes-objects'></a>

## Classes and objects

---

In python, everything is an "object" of some type. This is the basis of what is known as **Object Oriented Programming (OOP)**.

A *class* is a type of object. You can think of a class definition as a sort of "blueprint" that specifies the construction of a new object when instantiated.

> **Note:** Knowing how to define and use classes is essential to programming python at an intermediate or advanced level. I will cover the basics here, which will help you understand how things like `LinearRegression` in sklearn work.


<a id='coding-lr'></a>

## Coding our own version of the sklearn `LinearRegression` class

---

By now you are familiar with the `LinearRegression` class in sklearn. We will walk through the re-creation of this class (albeit a simplified version).


<a id='starting-class'></a>
### 1. Starting a basic python class

Below is the beginning of our class blueprint:

In [3]:
# A:

What are the components of this?

**`class`**

- The `class` is like `def`, but instead of defining a function it defines a class.

**`object`**

- `object` in the parentheses of the class definition indicate that this class "inherits" from the `object` class. The object class is a very general, very fundamental class in python. Inheritance means that whatever properties and function are part of the `object` class are passed down to our `SimpleLinearRegression` class.

**`def __init__(self)`**

- The `def __init__(self):` is our class's initialization function. This function is called when you instantiate the class by typing `SimpleLinearRegression()`

**`self`**

- `self` is the first argument to class definitions. It is a variable that refers to the **current instantiation of the class**. What does this mean? When you instantiate a class and assign it to a variable with `slr = SimpleLinearRegression()`, the `self` argument is now a reference to the current instantiation of the class `slr`. Now, when you use a function that is part of the class, it knows to use that specific object's function. This lets you have multiple instantiations of a class with the same function name.

**class attributes**

- `self.coef_` and `self.intercept_`, likewise, are "attributes" (variables) that are connected to the instantiation of the class. When self becomes `slr`, for example, the `self` becomes `slr` and `self.coef_` becomes `slr.coef`

In [4]:
# Inheritance example

---

<a id='class-function'></a>
### 2. Adding a class function

Now, just like with `__init__`, we can add functions to the class.

**Let's add a `fit()` method that will calculate the coefficients for a linear regression.**
- The function should have arguments `self`, `X` and `y`.
- Use the linear algebra equations above to calculate the coefficients and intercept.
- Assign the coefficients to `self.coef_` and the intercept to `self.intercept_`.

In [5]:
# A:

Notice how we assigned `self.coef_` inside of the `fit()` function.

This will set the class attribute `self.coef_`, and this attribute can be accessed by _any other function in the class without passing it as an argument!_

It can also be accessed by you after instantiating the class.

---

<a id='init-args'></a>
### 3. Assigning attributes during instantiation

There is an issue here - we may pass an `X` matrix in without an intercept. 

**Add a keyword argument to the `__init__` function which will specify whether the `X` matrix should have an intercept added or not.**

In [6]:
# A:

**Now, if we instantiate the class, it will assign `fit_intercept` to the class attribute `fit_intercept`. Try it out:**

In [7]:
# A:

---

<a id='intercept-adder'></a>
### 4. Add a function to add an intercept to the `X` matrix if necessary

This function will be called from inside the `fit` function and run conditional on the value of `self.fit_intercept`.

In [8]:
# A:

---

<a id='instantiate'></a>
### 5. Instantiate the class

At this point we can try out our class. 

**Instantiate the class and try out the coefficient fitting function on the housing data.**

In [9]:
# A:

Like in the sklearn `LinearRegression` class, we now have access to the assigned `coef_` and `intercept_` attributes after fitting the model.

---

<a id='predict'></a>
### 6. Add the `predict` function.

Let's add some more of the class methods that are in the real `LinearRegression` class.

**First off, add the `predict` function. It will take a design matrix `X` and return predictions for those rows.**

In [10]:
# A:

**Test out the predict function.**

In [11]:
# A:

---

<a id='score'></a>
### 7. Add a `score` function

This will calculate the $R^2$ of your model on a provided `X` and `y`.

> **Note:** You'll probably want to write a helper function to calculate the sum of squared errors, since this will be run for both the baseline model and the regression model in order to calculate the $R^2$.

In [12]:
# A:

<a id='verify'></a>

## Verify your class against the sklearn `LinearRegression` implementation.

---

Our class should return the same results for the $R^2$

In [13]:
# A:

<a id='inspection'></a>

## Inspecting a class

---

When we want to know more about a class object, we can use the "inspect" module. Specifically the `inspect.getmembers()` function takes an instantiated class as an argument and returns an information dictionary.

This can be helpful to know what attributes and methods are avaiable and basically, the blueprint of a class object in memory.  Depending on the way the class was implemented, you can usually find useful information hiding inside of `slr.__class__.__dict__` -- which can be easier to look at.  The "right way" is to use the "inspect" module.

In [14]:
import inspect

In [15]:
# A:

<a id='special'></a>

## Some special class methods

---

|Method| Description|
|--|--|
|\_\_init\_\_ ( self [,args...] )| Constructor (with any optional arguments) Sample Call : obj = className(args)
|\_\_del\_\_( self ) | Destructor, deletes an object Sample Call : del obj
|\_\_repr\_\_( self ) | Evaluatable string representation Sample Call : repr(obj)
|\_\_str\_\_( self ) | Printable string representation Sample Call : str(obj)
|\_\_cmp\_\_ ( self, x ) | Object comparison Sample Call : cmp(obj, x)

The `__repr__` function reports back something descriptive about what the class represents.  You can basically do whatever you want with it but the purpose of it is to convey something descirptive about your class.

The `__del__` method is the bookend function of `__init__`. You can use it to run code once your class is done executing.  

Generally it works well but in practice there are a few things watch out for.  Read more about [safely using Python destructors](http://eli.thegreenplace.net/2009/06/12/safely-using-destructors-in-python)